In [1]:
import numpy as np
import pandas as pd
import pickle
import requests
import tensorflow as tf
from transformers import TFAutoModel

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# url='https://kkb-production.jupyter-proxy.kaggle.net/k/144276518/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2IiwidHlwIjoiSldUIn0..ULwgZuiYrNHdG49vGuLOtg.HVwe4ekDMydvzj-XS6CzAwE77JcaPPMRV7ljPJA2K1U7d-qQl__yRfX6O3HyjQlJztfpTWl4kAR-lPU3-XJOeK8DxdaFJ2Uihr_9U3DokuL1HToOrIjWJDBxkDsq-pRGGA4FhntAg2nqlbDUklvmibrOl5ekpcFBFe6i_nKm7d2I3ejvHELacplt9B2b4KmoX9UUfICqxaRPNjcfqy-RVg.0OlO5P-NHl7jwtns4Agutw/proxy/files/BiomedVLP-CXR-BERT-general-vl_inputs.pkl?_xsrf=2%7C41122b45%7C7e453d9ebd4c16961b05a8c35be21bf4%7C1693419082'
# r = requests.get(url, allow_redirects=True)

# open('x_val.pkl', 'wb').write(r.content)

In [4]:
# with open('', 'wb') as f:
#     pickle.dump(tr_inputs, f)

In [2]:
with open('/kaggle/working/x_test.pkl', 'rb') as f:
    x_test = pickle.load(f)
# with open('/kaggle/working/x_train.pkl', 'rb') as f:
#     x_tr = pickle.load(f)
# with open('/kaggle/working/x_val.pkl', 'rb') as f:
#     x_val = pickle.load(f)

In [3]:
with open('/kaggle/input/y-train-test-val/y_test/kaggle/working/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)
# with open('/kaggle/input/y-train-test-val/y_tr/kaggle/working/y_tr.pkl', 'rb') as f:
#     y_tr = pickle.load(f)
# with open('/kaggle/input/y-train-test-val/y_val/kaggle/working/y_val.pkl', 'rb') as f:
#     y_val = pickle.load(f)

In [5]:
tr_ds = tf.data.Dataset.from_tensor_slices((x_tr['input_ids'], x_tr['attention_mask'], y_tr))
val_ds = tf.data.Dataset.from_tensor_slices((x_val['input_ids'], x_val['attention_mask'], y_val))

In [6]:
def map_func(input_ids, masks, labels):
    # we convert our three-item tuple into a two-item tuple where the input item is a dictionary
    return {'input_ids': input_ids, 'attention_mask': masks}, labels

# then we use the dataset map method to apply this transformation
tr_ds = tr_ds.map(map_func)
val_ds = val_ds.map(map_func)

In [7]:
BATCH_SIZE = 64
tr_ds = tr_ds.batch(BATCH_SIZE, drop_remainder=True)
val_ds = val_ds.batch(BATCH_SIZE, drop_remainder=True)

In [10]:
bert = TFAutoModel.from_pretrained("microsoft/BiomedVLP-CXR-BERT-general",from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['bert.embeddings.position_ids', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [8]:
N_EPOCHS = 4
LR = 0.001
DECAY = 1e-06
NUM_LABELS = 10

In [12]:
# two input layers, we ensure layer name variables match to dictionary keys in TF dataset
input_ids = tf.keras.layers.Input(shape=(512,), name='input_ids', dtype='int32')
mask = tf.keras.layers.Input(shape=(512,), name='attention_mask', dtype='int32')
bertmodel = bert.bert(input_ids, attention_mask=mask)[1]
drp = tf.keras.layers.Dropout(0.1)(bertmodel)
ln1 = tf.keras.layers.Dense(32, activation='linear')(drp)
bn1 = tf.keras.layers.BatchNormalization()(ln1)
rl1 = tf.keras.layers.Dense(64, activation='relu')(bn1)
ln2 = tf.keras.layers.Dense(32, activation='linear')(rl1)
bn1 = tf.keras.layers.BatchNormalization()(ln2)
rl2 = tf.keras.layers.Dense(64, activation='relu')(bn1)
y = tf.keras.layers.Dense(NUM_LABELS, activation='sigmoid', name='outputs')(rl2)

In [13]:
# initialize model
model = tf.keras.Model(inputs=[input_ids, mask], outputs=y)

# freeze bert layer
model.layers[2].trainable = False

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 512)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  109482240   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 512,                                           

In [14]:
optimizer = tf.keras.optimizers.AdamW(learning_rate=LR,weight_decay=DECAY)

In [15]:
loss = tf.keras.losses.BinaryCrossentropy()
metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall(),tf.keras.metrics.CategoricalAccuracy()]

In [16]:
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
#from tensorflow import ConfigProto
#from tensorflow import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [18]:
history = model.fit(
    tr_ds,
    validation_data=val_ds,
    epochs=N_EPOCHS
)

Epoch 1/4
1151/1151 [==============================] - 1873s 2s/step - loss: 0.5339 - precision: 0.5315 - recall: 0.1410 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3607 - val_loss: 0.5112 - val_precision: 0.5856 - val_recall: 0.1595 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.4103
Epoch 2/4
1151/1151 [==============================] - 1818s 2s/step - loss: 0.5216 - precision: 0.5645 - recall: 0.1596 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3803 - val_loss: 0.5095 - val_precision: 0.5804 - val_recall: 0.1743 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.4252
Epoch 3/4
1151/1151 [==============================] - 1859s 2s/step - loss: 0.5194 - precision: 0.5669 - recall: 0.1637 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3842 - val_loss: 0.5053 - val_precision: 0.5762 - val_recall: 0.2015 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.4149
Epoch 4/4
1151/1151 [==============================] - 1859s 2s/step - loss: 0.5180 - precision: 

In [19]:
model.save('BiomedVLP-CXR-BERT-general-4')

In [21]:
print("train loss:",history.history['loss'])
print("val loss:",history.history['val_loss'])

train loss: [0.5339366793632507, 0.5215806365013123, 0.5194080471992493, 0.5180252194404602]
val loss: [0.5111503005027771, 0.5094810724258423, 0.5053397417068481, 0.5071488618850708]


In [ ]:
# with open('/kaggle/working/BiomedNLP-PubMedBERT-base-uncased-abstract-4-pkl', 'rb') as f:
#     model = pickle.load(f,custom_objects={"TFBertModel": transformers.TFBertModel})

In [21]:
#model = load_model('/kaggle/working/BiomedNLP-PubMedBERT-base-uncased-abstract-4',custom_objects={"TFBertModel": TFBertModel})

In [22]:
history = model.fit(
    tr_ds,
    validation_data=val_ds,
    epochs=N_EPOCHS
)

Epoch 1/4
1151/1151 [==============================] - 1818s 2s/step - loss: 0.5170 - precision: 0.5718 - recall: 0.1728 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3863 - val_loss: 0.5059 - val_precision: 0.5600 - val_recall: 0.2375 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.4325
Epoch 2/4
1151/1151 [==============================] - 1859s 2s/step - loss: 0.5163 - precision: 0.5740 - recall: 0.1753 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3803 - val_loss: 0.5050 - val_precision: 0.5657 - val_recall: 0.2268 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.4349
Epoch 3/4
1151/1151 [==============================] - 1860s 2s/step - loss: 0.5158 - precision: 0.5736 - recall: 0.1755 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3833 - val_loss: 0.5062 - val_precision: 0.5706 - val_recall: 0.2008 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.4429
Epoch 4/4
1151/1151 [==============================] - 1859s 2s/step - loss: 0.5158 - precision: 

In [23]:
model.save('BiomedVLP-CXR-BERT-general-8')

In [24]:
with open('BiomedVLP-CXR-BERT-general-8-pkl', 'wb') as f:
    pickle.dump(model, f)

In [10]:
history = model.fit(
    tr_ds,
    validation_data=val_ds,
    epochs=N_EPOCHS
)

Epoch 1/4
1151/1151 [==============================] - 1876s 2s/step - loss: 0.5152 - precision: 0.5737 - recall: 0.1783 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3801 - val_loss: 0.5032 - val_precision: 0.5682 - val_recall: 0.2267 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.4338
Epoch 2/4
1151/1151 [==============================] - 1860s 2s/step - loss: 0.5148 - precision: 0.5750 - recall: 0.1793 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3800 - val_loss: 0.5064 - val_precision: 0.5743 - val_recall: 0.1929 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.4392
Epoch 3/4
1151/1151 [==============================] - 1819s 2s/step - loss: 0.5145 - precision: 0.5746 - recall: 0.1796 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3814 - val_loss: 0.5025 - val_precision: 0.5850 - val_recall: 0.1941 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.4284
Epoch 4/4
1151/1151 [==============================] - 1819s 2s/step - loss: 0.5142 - precision: 

In [11]:
model.save('BiomedVLP-CXR-BERT-general-12')

In [12]:
with open('BiomedVLP-CXR-BERT-general-12-pkl', 'wb') as f:
    pickle.dump(model, f)

In [21]:
history = model.fit(
    tr_ds,
    validation_data=val_ds,
    epochs=4
)

Epoch 1/4
1151/1151 [==============================] - 1820s 2s/step - loss: 0.5139 - precision: 0.5790 - recall: 0.1830 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3791 - val_loss: 0.5010 - val_precision: 0.5798 - val_recall: 0.2267 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.4210
Epoch 2/4
1151/1151 [==============================] - 1860s 2s/step - loss: 0.5139 - precision: 0.5765 - recall: 0.1841 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3810 - val_loss: 0.5023 - val_precision: 0.5783 - val_recall: 0.2136 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.4284
Epoch 3/4
1151/1151 [==============================] - 1860s 2s/step - loss: 0.5136 - precision: 0.5761 - recall: 0.1831 - accuracy: 0.0000e+00 - categorical_accuracy: 0.3818 - val_loss: 0.5026 - val_precision: 0.5879 - val_recall: 0.1977 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.4355
Epoch 4/4
1151/1151 [==============================] - 1820s 2s/step - loss: 0.5135 - precision: 

In [ ]:
print(history.history['loss'])
print(history.history['val_loss'])

In [22]:
model.save('BiomedVLP-CXR-BERT-general-16')
with open('BiomedVLP-CXR-BERT-general-16-pkl', 'wb') as f:
    pickle.dump(model, f)

Prediction

In [4]:
from tensorflow.keras.models import load_model
from transformers import TFBertModel
# model_1 = load_model('BiomedVLP-CXR-BERT-general-4',custom_objects={"TFBertModel": TFBertModel})
# model_2 = load_model('BiomedVLP-CXR-BERT-general-8',custom_objects={"TFBertModel": TFBertModel})
# model_3 = load_model('BiomedVLP-CXR-BERT-general-12',custom_objects={"TFBertModel": TFBertModel})
model_4 = load_model('BiomedVLP-CXR-BERT-general-16',custom_objects={"TFBertModel": TFBertModel})

In [5]:
# probes_1 = model_1.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
# probes_2 = model_2.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
# probes_3 = model_3.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
probes_4 = model_4.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})

640/640 [==============================] - 438s 676ms/step


In [6]:
cols = ['I10', 'E785', 'Z87891', 'K219', 'F329', 'I2510', 'F419', 'N179',
       'Z794', 'E039']

In [7]:
with open('16-predictions', 'wb') as f:
    pickle.dump(probes_4, f)

In [7]:
threshold  = np.arange(0.1,0.9,0.01)
threshold

array([0.1 , 0.11, 0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 ,
       0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31,
       0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42,
       0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53,
       0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64,
       0.65, 0.66, 0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75,
       0.76, 0.77, 0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86,
       0.87, 0.88, 0.89])

In [8]:
import numpy as np
from sklearn.metrics import roc_curve, precision_score, recall_score, f1_score

# Example predicted labels (probabilities)
predicted_labels = probes_4
# True labels
true_labels = y_test

# Determine threshold for each label
thresholds = []
for label_id in range(predicted_labels.shape[1]):
    # Get probabilities for the current label
    label_probs = predicted_labels[:, label_id]
    # Get true labels for the current label
    label_true = true_labels[:, label_id]
    # Calculate the false positive rate, true positive rate, and thresholds
    fpr, tpr, roc_thresholds = roc_curve(label_true, label_probs)
    # Calculate the distances from the top-left corner (0, 1)
    distances = np.sqrt(np.square(1 - tpr) + np.square(fpr))
    # Find the index of the threshold that minimizes the distance
    best_index = np.argmin(distances)
    # Get the corresponding threshold
    best_threshold = roc_thresholds[best_index]
    thresholds.append(best_threshold)
# Print the thresholds for each label
for label_id, threshold in enumerate(thresholds):
    print(f"Label {label_id + 1}: {threshold}")

Label 1: 0.4852258563041687
Label 2: 0.45131203532218933
Label 3: 0.3432055413722992
Label 4: 0.2905704975128174
Label 5: 0.2269510179758072
Label 6: 0.179512619972229
Label 7: 0.17128156125545502
Label 8: 0.18445748090744019
Label 9: 0.13755233585834503
Label 10: 0.1571851372718811


In [9]:
binary_predictions = (predicted_labels >= thresholds).astype(int)

In [10]:
from sklearn.metrics import classification_report
print("Classification_report:")
print(classification_report(y_test,binary_predictions,target_names=cols,zero_division=0))

Classification_report:
              precision    recall  f1-score   support

         I10       0.56      0.66      0.61      8790
        E785       0.57      0.69      0.62      8848
      Z87891       0.45      0.62      0.52      7418
        K219       0.38      0.62      0.47      6204
        F329       0.32      0.63      0.42      4676
       I2510       0.44      0.74      0.55      4595
        F419       0.29      0.64      0.40      3859
        N179       0.35      0.70      0.46      3932
        Z794       0.30      0.69      0.42      3139
        E039       0.22      0.64      0.33      3018

   micro avg       0.40      0.66      0.50     54479
   macro avg       0.39      0.66      0.48     54479
weighted avg       0.43      0.66      0.51     54479
 samples avg       0.40      0.65      0.46     54479

